**Objective**: Process the raw json data I have collected into SQLite databases

**Author**: Matthew Thoomkuzhy

**Date Last edited**: 01/12/2024

---
# Data Processing
This notebook contains the SQLite database I have processed from my JSON data, I will be cleaning the JSON data to create 3 dataframes:

* df_artists_general_data: Stores metadata and general info about top 100 artists

* df_artists_related_data: Stores data about the related t100 artists of each t100 artist

* df_top_tracks: Stores data about the top tracks of each top 100 artist




importing relevant modules:

In [1]:
import sqlalchemy
import requests
import os
import base64
from requests import post
import json
import pandas as pd
from IPython.display import display
from IPython.display import Image
from sqlalchemy import create_engine
from sqlalchemy import inspect 
from sqlalchemy import text 
from datetime import datetime
top_100_artists = [
    "Taylor Swift",
    "The Weeknd",
    "Drake",
    "Bad Bunny",
    "Ed Sheeran",
    "Justin Bieber",
    "Ariana Grande",
    "Eminem",
    "Billie Eilish",
    "Post Malone",
    "BTS",
    "J Balvin",
    "Kanye West",
    "Rihanna",
    "Doja Cat",
    "Olivia Rodrigo",
    "Dua Lipa",
    "Travis Scott",
    "Kendrick Lamar",
    "Lil Nas X",
    "Shawn Mendes",
    "Harry Styles",
    "Maroon 5",
    "Imagine Dragons",
    "Cardi B",
    "Juice WRLD",
    "Lil Baby",
    "SZA",
    "Future",
    "The Kid LAROI",
    "Badshah",
    "ANITTA",
    "Camila Cabello",
    "Selena Gomez",
    "Halsey",
    "Nicki Minaj",
    "Lil Wayne",
    "21 Savage",
    "Megan Thee Stallion",
    "Khalid",
    "Lizzo",
    "Billie Eilish",
    "J. Cole",
    "Adele",
    "Bruno Mars",
    "Chris Brown",
    "Marshmello",
    "DJ Snake",
    "Daddy Yankee",
    "Maluma",
    "Ozuna",
    "Karol G",
    "Becky G",
    "Nicky Jam",
    "Sech",
    "Myke Towers",
    "Rauw Alejandro",
    "Farruko",
    "Jhay Cortez",
    "Lunay",
    "Tainy",
    "Arcangel",
    "Bryant Myers",
    "De La Ghetto",
    "Yandel",
    "Wisin",
    "Zion & Lennox",
    "Natti Natasha",
    "Manuel Turizo",
    "Reik",
    "CNCO",
    "Sebastián Yatra",
    "Morat",
    "Piso 21",
    "Mau y Ricky",
    "Camilo",
    "Kany García",
    "Ricardo Arjona",
    "Alejandro Sanz",
    "Pablo Alborán",
    "Luis Fonsi",
    "Enrique Iglesias",
    "Shakira",
    "Thalía",
    "Paulina Rubio",
    "Gloria Trevi",
    "Alejandra Guzmán",
    "LA INDIA",
    "Ivy Queen",
    "Celia Cruz",
    "Marc Anthony",
    "Romeo Santos",
    "Prince Royce",
    "Aventura",
    "Gente de Zona",
    "Chino & Nacho",
    "Wisin & Yandel",
    "Plan B",
    "Jowell & Randy",
    "Alexis & Fido"
]  


---

### 1. T100_artists_data:
I will begin processing the raw json for the general top 100 artist into a pandas dataframe then an SQLite database

In [2]:

def T100_gen_data(json_file_path):

    with open(json_file_path, 'r') as file:
        data = json.load(file)

    # Extract relevant data into a list of dictionaries
    artists_data = []
    for artist_name, artist_info in data.items():
        artist_items = artist_info.get("artists", {}).get("items", [])
        if artist_items:
            artist_details = artist_items[0]
            artists_data.append({
                "Artist_ID": artist_details.get("id"),
                "Artist_Name": artist_details.get("name"),
                "Followers": artist_details.get("followers", {}).get("total", 0),
                "Genres": artist_details.get("genres", []),  # Keep genres as a list
                "Popularity": artist_details.get("popularity")
            })

    # Convert the extracted data into a Pandas DataFrame
    df = pd.DataFrame(artists_data)

    # Explode the 'genres' column to normalize
    df = df.explode('Genres')

    # Rename 'genres' column to 'genre' for clarity
    df.rename(columns={'Genres': 'Genre'}, inplace=True)

    return df


# Obtaining table
json_file_path = '../data/raw/T100_artists_data.json'
df_artists_general_data = T100_gen_data(json_file_path)
df_artists_general_data = df_artists_general_data.dropna()
df_artists_general_data


,Artist_ID,Artist_Name,Followers,Genre,Popularity
0,06HL4z0CvFAxyc27GXpf02,Taylor Swift,126653193,pop,100
1,1Xyo4u8uXC1ZmMpatF05PJ,The Weeknd,94962172,canadian contemporary r&b,96
1,1Xyo4u8uXC1ZmMpatF05PJ,The Weeknd,94962172,canadian pop,96
1,1Xyo4u8uXC1ZmMpatF05PJ,The Weeknd,94962172,pop,96
2,3TVXtAsR1Inumwj472S9r4,Drake,93637851,canadian hip hop,96
...,...,...,...,...,...
96,2jSGzJw0ebJLu7OLVSOcBP,Plan B,5235698,trap latino,81
96,2jSGzJw0ebJLu7OLVSOcBP,Plan B,5235698,urbano latino,81
97,4IMAo2UQchVFyPH24PAjUs,Jowell & Randy,4432125,reggaeton,72
97,4IMAo2UQchVFyPH24PAjUs,Jowell & Randy,4432125,urbano latino,72


Now I am going to turn the above table regarding the general artist information into an SQlite database:

---

### 2. T100 related artists data:
I am going to be obtaining a table which stores the following information:

* Related T100 artists of T100 Artists
* Number of related artists
* Number of related T100 artists

In [3]:

def T100_related_data(json_file_path):

    with open(json_file_path, 'r') as file:
        data = json.load(file)

    # Extract relevant data into a list of dictionaries
    artists_data = []
    for artist_name, artist_info in data.items():
        artists_data.append({
            "Artist_ID": artist_info.get("artist_id"),
            "Artist_Name": artist_name,
            "Related_T100_Artist": artist_info.get("matches_with_top_100", []),  # Keep list as is
        })

    # Convert the extracted data into a Pandas DataFrame and explode the related artist column (previously a list)
    df = pd.DataFrame(artists_data).explode('Related_T100_Artist')
    return df

# Obtaining table
json_file_path = '../data/raw/T100_related_artists_count.json'
df_artists_related_data = T100_related_data(json_file_path)
df_artists_related_data = df_artists_related_data.dropna()
df_artists_related_data


,Artist_ID,Artist_Name,Related_T100_Artist
0,06HL4z0CvFAxyc27GXpf02,Taylor Swift,Olivia Rodrigo
0,06HL4z0CvFAxyc27GXpf02,Taylor Swift,Ariana Grande
0,06HL4z0CvFAxyc27GXpf02,Taylor Swift,Billie Eilish
0,06HL4z0CvFAxyc27GXpf02,Taylor Swift,Harry Styles
0,06HL4z0CvFAxyc27GXpf02,Taylor Swift,Shawn Mendes
...,...,...,...
96,2jSGzJw0ebJLu7OLVSOcBP,Plan B,Ozuna
96,2jSGzJw0ebJLu7OLVSOcBP,Plan B,Zion & Lennox
96,2jSGzJw0ebJLu7OLVSOcBP,Plan B,Bad Bunny
97,4IMAo2UQchVFyPH24PAjUs,Jowell & Randy,De La Ghetto


---

### 3. T100 Artists top tracks and their features

I will be obtaining a normalized dataframe containing the following information:

* track's of top 100 artists
* duration of tracks
* whether the tracks are explicit

In [4]:
import json
import pandas as pd

def top_tracks(json_file_path, top_100_artists):

    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # Extract relevant data into a list of dictionaries
    tracks_data = []
    for artist_name, artist_info in data.items():
        for track in artist_info.get("tracks", []):
            tracks_data.append({
                "Track_ID": track.get("id"),
                "Track_Name": track.get("name"),
                "Artist_Name": [artist["name"] for artist in track.get("artists", [])],  # Keep as a list
                "Duration": track.get("duration_ms", 0),
                "Explicit": track.get("explicit", False)
            })
    
    df = pd.DataFrame(tracks_data)
    # Normalize the Artists column
    df = df.explode('Artist_Name')
    # Filter to keep only rows where the artist is in the top 100 artists
    df = df[df['Artist_Name'].isin(top_100_artists)]

    return df

json_file_path = '../data/raw/T100_artists_top_tracks.json'

# Obtain dataframe
df_top_tracks = top_tracks(json_file_path, top_100_artists)
df_top_tracks = df_top_tracks.dropna()
df_top_tracks

,Track_ID,Track_Name,Artist_Name,Duration,Explicit
0,1BxfuPKGuaTgP7aM0Bbdwr,Cruel Summer,Taylor Swift,178426,False
1,2OzhQlSqBEmt7hmkYxfT6m,Fortnight (feat. Post Malone),Taylor Swift,228965,False
1,2OzhQlSqBEmt7hmkYxfT6m,Fortnight (feat. Post Malone),Post Malone,228965,False
2,4q5YezDOIPcoLr8R81x9qy,I Can Do It With a Broken Heart,Taylor Swift,218004,True
3,3hUxzQpSfdDqwM3ZTFQY0K,august,Taylor Swift,261922,False
...,...,...,...,...,...
976,1WeBMnIHSyCHB63dOzbE5p,X-100,Jowell & Randy,119879,True
977,0ySExWMtzlVKO70rPYEz84,Vamo a Busal,Jowell & Randy,240000,False
978,5bfNJD8roiR6dNRYGZGWvz,Hey Mister - Remix,Jowell & Randy,248695,False
979,2MM8KVLGV6oHwuTmOWQg5Y,Sin Panty Maria,Jowell & Randy,192928,True


---

## Creating SQLite database

Creating a databse which includes takes information from the 3 dataframes to create 5 tables:

1. artist_metadata:  stores Artist ID, artist name, followers and Popularity

2. artist_genres: stores the genres of each artist

3. artist_relationships: stores the relationships of each artist

4. artist_tracks: stores track data for top 100  most listened to tracks of 2023

5. track_artist: database to facilitate the many to many relationship between artist and tracks


In [5]:
from sqlalchemy import create_engine, text
import pandas as pd

# Create the database engine
engine = create_engine('sqlite:///../data/database/artist.db')  

# Create tables
create_artist_metadata = text("""
CREATE TABLE IF NOT EXISTS artist_metadata (
    Artist_ID VARCHAR(255) PRIMARY KEY,
    Artist_Name VARCHAR(255) NOT NULL,
    Followers INTEGER,
    Popularity INTEGER
);
""")

create_artist_genres = text("""
CREATE TABLE IF NOT EXISTS artist_genres (
    Artist_ID VARCHAR(255),
    Genre VARCHAR(255),
    PRIMARY KEY (Artist_ID, Genre),
    FOREIGN KEY (Artist_ID) REFERENCES artist_metadata(Artist_ID)
);
""")

create_artist_relationships = text("""
CREATE TABLE IF NOT EXISTS artist_relationships (
    Artist_ID VARCHAR(255),
    Related_T100_Artist TEXT,
    PRIMARY KEY (Artist_ID, Related_T100_Artist),
    FOREIGN KEY (Artist_ID) REFERENCES artist_metadata(Artist_ID)
);
""")

# Updated schema for artist_tracks without Artist_Name
create_artist_tracks = text("""
CREATE TABLE IF NOT EXISTS artist_tracks (
    Track_ID VARCHAR(255) PRIMARY KEY,
    Track_Name TEXT NOT NULL,
    Duration INT NOT NULL,
    Explicit BOOLEAN NOT NULL
);
""")

# track_artists table for many-to-many relationships
create_track_artists = text("""
CREATE TABLE IF NOT EXISTS track_artists (
    Track_ID VARCHAR(255),
    Artist_ID VARCHAR(255),
    PRIMARY KEY (Track_ID, Artist_ID),
    FOREIGN KEY (Track_ID) REFERENCES artist_tracks(Track_ID),
    FOREIGN KEY (Artist_ID) REFERENCES artist_metadata(Artist_ID)
);
""")

# Drop and recreate tables
with engine.connect() as conn:
    conn.execute(text('DROP TABLE IF EXISTS track_artists;'))
    conn.execute(text('DROP TABLE IF EXISTS artist_relationships;'))
    conn.execute(text('DROP TABLE IF EXISTS artist_genres;'))
    conn.execute(text('DROP TABLE IF EXISTS artist_metadata;'))
    conn.execute(text('DROP TABLE IF EXISTS artist_tracks;'))
    conn.execute(create_artist_metadata)
    conn.execute(create_artist_genres)
    conn.execute(create_artist_relationships)
    conn.execute(create_artist_tracks)
    conn.execute(create_track_artists)

# Insert data into the tables

# artist_metadata
artist_metadata = df_artists_general_data[['Artist_ID', 'Artist_Name', 'Followers', 'Popularity']].drop_duplicates()
artist_metadata.to_sql('artist_metadata', con=engine, if_exists='append', index=False)

# artist_genres
artist_genres = df_artists_general_data[['Artist_ID', 'Genre']].drop_duplicates()
artist_genres.to_sql('artist_genres', con=engine, if_exists='append', index=False)

# artist_relationships
artist_relationships = df_artists_related_data[['Artist_ID', 'Related_T100_Artist']].drop_duplicates()
artist_relationships.to_sql('artist_relationships', con=engine, if_exists='append', index=False)

# artist_tracks: Insert tracks without Artist_Name
tracks_metadata = df_top_tracks[['Track_ID', 'Track_Name', 'Duration', 'Explicit']].drop_duplicates()
tracks_metadata.to_sql('artist_tracks', con=engine, if_exists='append', index=False)

# track_artists: Add artist-to-track relationships

# Standardize Artist_Name in both DataFrames
df_top_tracks['Artist_Name'] = df_top_tracks['Artist_Name'].str.strip().str.lower()
artist_metadata['Artist_Name'] = artist_metadata['Artist_Name'].str.strip().str.lower()

# Remove rows with null Artist_Name in df_top_tracks
df_top_tracks = df_top_tracks.dropna(subset=['Artist_Name'])

# Merge Artist_Name from df_top_tracks with Artist_ID from artist_metadata
df_top_tracks = df_top_tracks.merge(
    artist_metadata.rename(columns={'Artist_ID': 'Metadata_Artist_ID'})[['Metadata_Artist_ID', 'Artist_Name']],
    on='Artist_Name',
    how='left'
)

# Verify merge result for unmatched rows
unmatched = df_top_tracks[df_top_tracks['Metadata_Artist_ID'].isna()]
if not unmatched.empty:
    print("Unmatched rows in df_top_tracks:", unmatched)

# Use the renamed Artist_ID column for track_artists table
track_artists = df_top_tracks[['Track_ID', 'Metadata_Artist_ID']].drop_duplicates()

# Rename back to Artist_ID before inserting into the database
track_artists = track_artists.rename(columns={'Metadata_Artist_ID': 'Artist_ID'})

# Insert into the database
track_artists.to_sql('track_artists', con=engine, if_exists='append', index=False)


Unmatched rows in df_top_tracks:                     Track_ID                              Track_Name  \
819   5anRovfNkeo83bshruaWX9                                 Soy Mia   
897   3eSJmGWqoBRx5wbFCtvPtz                    Color Esperanza 2020   
912   5EbtodsuLbxrFDH6j5avVS       El Mismo Aire - con Pablo Alborán   
924   14k3Rb7Xgte9fsndjdenmq                         Te Lo Agradezco   
925   44kI7aJvJAYJkREVk0Rmei                                Confieso   
926   3w6HBymB6v99gmhz4aiX1g                            Para Siempre   
927   6lPWLimbdg7Wbf9ncCVE8O                            La Siguiente   
928   4wJXlFV0oh88ktxE54uqKE                     DPM (De Pxta Madre)   
929   18nR6OK2Kj0CxRcaWbpOF6                            En Esta Boca   
930   5FCfMmtZXO5GoJrUtpUsAU                                   Duelo   
931   58zE9NyMKGTpzV5J8WyXXe                           Hoy Ya Me Voy   
932   03Rj7wKcBWBT444nqDRVHa                                  García   
937   3eSJmGWqoBRx5wbFCtvPtz   

1032